In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#dependencies
import warnings
warnings.filterwarnings("ignore")
!pip install contractions
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
import re
import pickle
import numpy as np
import pandas as pd
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPooling1D, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import gensim
from sklearn.metrics import confusion_matrix
import tensorflow  as tf 
from tensorflow.keras.models import Model, load_model
#from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
#from tokenizers import Tokenizer
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow.keras

In [ ]:
df_2 = pd.read_csv("/kaggle/input/fake-news/train.csv", header=0, index_col=0)
df_t = pd.read_csv("/kaggle/input/fake-news/test.csv", header=0, index_col=0)
df_2 = df_2.drop(['title','author'], axis = 1)
df_t = df_t.drop(['title','author'], axis = 1)
df_2.dropna(inplace = True)
df_t.fillna('',inplace = True)
#print(df_2.isnull().sum(axis = 0))



In [ ]:
def clean_text(text_col ):
    text_col = text_col.apply(lambda x: [contractions.fix(word, slang=False).lower() for word in x.split()])
    text_col = text_col.apply(lambda x: [re.sub(r'[^\w\s]','', word)  for word in x])
    stop_words = set(stopwords.words('english'))
    text_col = text_col.apply(lambda x: [word for word in x if word not in stop_words])
    text_col = text_col.apply(lambda x: [word for word in x if re.search("[@_!#$%^&*()<>?/|}{~:0-9]", word) == None])
    return text_col
df_2["text"] = clean_text(df_2["text"])
df_t["text"] = clean_text(df_t["text"])
df_2['label'] = df_2['label'].apply(lambda x: int(x))
y = df_2['label']

In [ ]:
#y = df_2["label"]

In [ ]:
#lemmatizing
wordnet_lemmatizer = WordNetLemmatizer()
x = []
x_test = []
english_words = set(nltk.corpus.words.words())
for words in df_2['text']:
        tmp = []
        fil_wor = [wordnet_lemmatizer.lemmatize(word, 'n') for word in words if word in english_words]
        tmp.extend(fil_wor)
        x.append(tmp)
        
for words in df_t['text']:
        tmp = []
        fil_wor = [wordnet_lemmatizer.lemmatize(word, 'n') for word in words if word in english_words]
        tmp.extend(fil_wor)
        x_test.append(tmp)
        
df_2["text"] = x
df_t["text"] = x_test


In [ ]:
#creating word embedding
x_all = x.copy()
for y in  range(len(x_test)):
    x_all.append(x_test[y])
#n of vectors we are generating
EMBEDDING_DIM = 100
#Creating Word Vectors by Word2Vec Method (takes time...)
w2v_model = gensim.models.Word2Vec(sentences=x_all, vector_size=EMBEDDING_DIM, window=5, min_count=1)
print(len(w2v_model.wv))

#testing a word embedding
w2v_model.wv["liberty"]

#similarity between words
word = 'people'
w2v_model.wv.most_similar(word)

In [ ]:
#tokenizing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

x = tokenizer.texts_to_sequences(x)
x_test = tokenizer.texts_to_sequences(x_test)
print(x[0][:10])

word_index = tokenizer.word_index
for word, num in word_index.items():
    print(f"{word} -> {num}")
    if num == 10:
        break

In [ ]:
#padding
maxlen = 700
x = pad_sequences(x, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [ ]:
# Function to create weight matrix from word2vec gensim model
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix, vocab_size

#Getting embedding vectors from word2vec and usings it as weights of non-trainable keras embedding layer
embedding_vectors, vocab_size = get_weight_matrix(w2v_model.wv, word_index)

In [ ]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False))
#LSTM 
model.add(Dropout(0.2))
#model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(units=128,dropout=0.2, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(units=128,dropout=0.2))
model.add(BatchNormalization())
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y)
model.fit(X_train, y_train, validation_data= (X_test,y_test), epochs=50)

In [ ]:
#validation_data_performance evaluation
y_pred = (model.predict(X_test) >= 0.5).astype("int")
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred)) 
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
#test_data_for_scoring_on_kaggle
y_t = (model.predict(x_test) >= 0.5).astype("int")
result = pd.DataFrame({"id" :df_t.index, "label":y_t.squeeze() }, index = None )
result.to_csv("result_rnn.csv",index = False)

In [ ]:
#let's include an attention layer in our model
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features)+ self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score),axis = 1)
        context_vector = attention_weights * features
        context_vector  = tf.reduce_sum(context_vector, axis = 1)
        return context_vector, attention_weights

In [ ]:
'''#dd attention layer to the deep learning network
class Attention(Layer):
    def __init__(self,**kwargs):
        super(Attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', #shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', #shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(Attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context'''

In [ ]:
#RNN with Attention model
sequence_input = Input(shape = (maxlen,), dtype = "int32")
embedding = Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=maxlen, trainable=False)(sequence_input)
dropout = Dropout(0.2)(embedding)

conv1 = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(dropout)
maxp = MaxPooling1D(pool_size=2)(conv1)
#(lstm, state_h, state_c) = LSTM(units=128,return_sequences=True,dropout=0.2, return_state= True)(maxp)
#bn1 = BatchNormalization()((lstm, state_h, state_c))
(lstm, state_h, state_c) = LSTM(units=128,dropout=0.2, return_sequences=True, return_state= True)(maxp)
context_vector, attention_weights = Attention(10)(lstm, state_h)
densee = Dense(20, activation='relu')(context_vector)
#bn = BatchNormalization()(densee)
dropout2 = Dropout(0.2)(densee)
densef = Dense(1, activation='sigmoid')(dropout2)
model = tensorflow.keras.Model(inputs = sequence_input, outputs = densef)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
display(model.summary())
model.fit(X_train, y_train, validation_data= (X_test,y_test), epochs=50)

In [ ]:
#checking accuracy on validation data
y_pred = (model.predict(X_test) >= 0.5).astype("int")
print(accuracy_score(y_test, y_pred))

In [ ]:
y_t = (model.predict(x_test) >= 0.5).astype("int")
result = pd.DataFrame({"id" :df_t.index, "label":y_t.squeeze() }, index = None )
result.to_csv("result_rnnattenion.csv",index = False)